# Create the dpl translations for RAG
Call dpl API, save translations to excel


In [ ]:
import deepl
import pandas as pd
import time

In [ ]:
# to use deepl you need an API from deepl
api_key = "api_key_here"

In [ ]:
# loading the list of crop descriptions to be translated
d_path = '/home/automate_eurocrops/data/raw/' 
sv_path = '/home/automate_eurocrops/data/processed'
sv_name = 'BW_trans.csv'
file_name = 'BW.csv'
data = pd.read_csv(d_path+file_name)
text = pd.DataFrame(data['original_name'])


In [ ]:
# processing in batch and with forced sleep delays as frequent api calls will lead to dpl rejecting the call

def translate_batch(texts, api_key, batch_size=5, delay_between_batches=10):
    translator = deepl.Translator(api_key)
    
    all_translations = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        batch_translations = []
        
        print(f"Processing batch {i//batch_size + 1} ({i+1}-{min(i+batch_size, len(texts))} of {len(texts)})")
        
        for text_val in batch:

            try:
                result = translator.translate_text(text_val, target_lang="EN-GB")
                batch_translations.append(result)
                time.sleep(0.5)  # Small delay between individual requests
            except Exception as e:
                print(f"Failed to translate '{text_val}': {e}")
                batch_translations.append(text_val)
        
        all_translations.extend(batch_translations)
        
        # Long pause between batches
        if i + batch_size < len(texts):
            print(f"Waiting {delay_between_batches}s before next batch...")
            time.sleep(delay_between_batches)
    
    return all_translations


In [ ]:
translated_texts = translate_batch(text['original_name'].tolist(), api_key, batch_size=5, delay_between_batches=10)
text['dpl_trans'] = translated_texts

In [ ]:
# saving the results
text.to_excel(sv_path+sv_name, index=False)